# Answering Business Questions Using SQL

The Chinook record store has just signed a deal with a new record label, and my objective is to select the first three albums that will be added to the store, from a list of four. The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA.

My goal is to find out which genres sell the best in the USA, the top 3 sales representatives of the company and the top 3 countries from which the company can make maximum sales. 

In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of Data

In [3]:
%%sql
SELECT name,
       type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * 
FROM invoice_line
LIMIT 4;

Done.


invoice_line_id,invoice_id,track_id,unit_price,quantity
1,1,1158,0.99,1
2,1,1159,0.99,1
3,1,1160,0.99,1
4,1,1161,0.99,1


In [4]:
%%sql
SELECT * 
FROM track
LIMIT 4;

Done.


track_id,name,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price
1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Hoffman",230619,3990994,0.99
4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. Dirkscneider & W. Hoffman",252051,4331779,0.99


In [5]:
%%sql
SELECT * 
FROM genre
LIMIT 4;

Done.


genre_id,name
1,Rock
2,Jazz
3,Metal
4,Alternative & Punk


## Selecting New Albums Genres to Purchase

Query that returns each genre, with the number of tracks sold in the USA in absolute numbers and in percentages.


In [10]:
%%sql
WITH usa_track_sale AS
       (
        SELECT track_id,
               SUM(quantity) AS qty 
        FROM invoice_line AS il
        INNER JOIN invoice AS i
            ON i.invoice_id = il.invoice_id
        INNER JOIN customer AS c
            ON c.customer_id = i.customer_id
        WHERE c.country = 'USA'
        group by 1
        ),
    
genre_sale AS (
        SELECT g.name AS genre_name, 
               SUM(qty) AS qty, 
               (SELECT SUM(qty) AS tot
                FROM usa_track_sale) AS tot
        FROM track AS t
        INNER JOIN usa_track_sale ts
            ON t.track_id = ts.track_id
        INNER JOIN genre g
            ON t.genre_id = g.genre_id
        GROUP BY 1    
        )

SELECT genre_name,
       qty AS tracks_sold, 
       cast(qty as float)/tot AS percentage_sold
from genre_sale
order by 2 desc
;

Done.


genre_name,tracks_sold,percentage_sold
Rock,561,0.5337773549000951
Alternative & Punk,130,0.12369172216936251
Metal,124,0.11798287345385347
R&B/Soul,53,0.05042816365366318
Blues,36,0.03425309229305423
Alternative,35,0.03330161750713606
Latin,22,0.02093244529019981
Pop,22,0.02093244529019981
Hip Hop/Rap,20,0.019029495718363463
Jazz,14,0.013320647002854425


Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

Red Tone (Punk)<br>
Slim Jim Bites (Blues)<br>
Meteor and the Girls (Pop)<br>

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so I recommend that we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

## Analyzing Employee Sales Performance

Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. Here, I am analyzing the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [13]:
%%sql
SELECT e.*
FROM employee e
    JOIN customer c
        ON e.employee_id = c.support_rep_id
limit 5        

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com


In [9]:
%%sql

WITH employee_level_sales AS (
    SELECT 
        e.first_name, 
        e.last_name,
        e.hire_date,
        SUM(total) AS total    
    FROM employee e
    JOIN customer c
        ON e.employee_id = c.support_rep_id
    JOIN invoice i
        ON i.customer_id = c.customer_id
    GROUP BY 1,2
    )

SELECT * FROM employee_level_sales limit 5;

Done.


first_name,last_name,hire_date,total
Jane,Peacock,2017-04-01 00:00:00,1731.5100000000039
Margaret,Park,2017-05-03 00:00:00,1584.0000000000034
Steve,Johnson,2017-10-17 00:00:00,1393.920000000002


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

## Analyzing Sales by Country

In particular, I am calculating the following for each country:

1.) Total number of customers <br>
2.) Total value of sales <br>
3.) Average value of sales per customer <br>
4.) Average order value <br>

Because there are a number of countries with only one customer, we can group these customers as "Other".

In [11]:
%%sql
select distinct quantity from invoice_line

Done.


quantity
1


In [10]:
%%sql

WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       i.*
     FROM invoice i 
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )

SELECT
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
FROM
    (
    SELECT
        country,
        count(distinct customer_id) customers,
        SUM(total) AS total_sales,
        SUM(total)/count(distinct customer_id) as customer_lifetime_value,
        avg(total) as average_order,      
        CASE
            WHEN country = "Other" THEN 1
            ELSE 0
        END AS sort
    FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC
    );

Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.4899999999998,7.942671755725189,80.0376923076923
Canada,8,535.5900000000001,7.047236842105265,66.94875000000002
Brazil,5,427.68000000000006,7.0111475409836075,85.53600000000002
France,5,389.0699999999999,7.781399999999998,77.81399999999998
Germany,4,334.62,8.161463414634147,83.655
Czech Republic,2,273.24000000000007,9.108000000000002,136.62000000000003
United Kingdom,3,245.52,8.768571428571429,81.84
Portugal,2,185.13000000000002,6.383793103448276,92.56500000000001
India,2,183.14999999999998,8.72142857142857,91.57499999999999
Other,15,1094.9400000000003,7.44857142857143,72.99600000000002


Based on the data, the top 3 countries from which the company can make the maximum sales are:

USA <br>
Canada <br>
Brazil <br>

Based on the data, there may be opportunity in the following countries:

Czech Republic <br>
United Kingdom <br>
India <br>


## Conclusion 

Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following genres: Punk, Blues and Pop.

However, these three genres only make up only 17% of total sales, so I recommend that we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.

Jane Peacock, Margaret Park and Steve Johnson are the top 3 sales support agents within the company.

Based on the data, there may be opportunity in the following countries:

Czech Republic <br>
United Kingdom <br>
India <br>

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.
